In [17]:
import acquire
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup 
import warnings
warnings.filterwarnings("ignore")

In [24]:
def get_url_list():
    '''
    Gets a list of urls from the github star ranking page, iterating through the different pages and grabbing,
    each url from the different github repositories.
    '''
    # Created an empty list that will hold all of the github links from each page.
    link_list = []
    # Created a for loop in order to get different pages for the p = {i}
    for i in range(1,99):
        # When 10 pages are accessed by the web scraper this is here to take a minute break to avoid,
        # having github throtal the webscrape.
        if i%10 == 0:
            # prints a different link in the link list to show that the pulls are working correctly
            print(link_list[i*2])
            # Prints the length of the link list to show that the list is growing correctly
            print(len(link_list))
            # shows that the sleep function is about to start and will pause function for 60 seconds
            print('waiting')
            sleep(60)
            # shows that the sleep is over
            print('continue')
            # Creates a response basically pulling in https://github.com/search?p=&q=stars%3A%3E0&s=stars&type=Repositories this link 99 times
            response = requests.get(f'https://github.com/search?p=100{i}&q=stars%3A%3E0&s=stars&type=Repositories',headers = {'user-agent': 'Codeup DS Germain'})
            # gets the text version to prepare it for beautiful soup
            html = response.text
            # use beautiful soup on the html
            soup = BeautifulSoup(html)
            # pulls the target html which holds the link of each github on that page
            text_with_link = soup.select('.v-align-middle')
            # iterates through each peice of html and will check if the regex below matches with anything.
            for link in text_with_link:
                # turning the html to a string
                link_ispy = str(link)
                # setting all the links to match
                match = re.search(r'(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-]\w+)', link_ispy)
                # checks if match is null and only passes through non null values
                if match is not None:
                    # appends the link to the link list
                    link_list.append(match.group(0))
        else:
            # Creates a response basically pulling in https://github.com/search?p=&q=stars%3A%3E0&s=stars&type=Repositories this link 99 times
            response = requests.get(f'https://github.com/search?p={i}&q=stars%3A%3E0&s=stars&type=Repositories',headers = {'user-agent': 'Codeup DS Germain'})
            # gets the text version to prepare it for beautiful soup
            html = response.text
            # use beautiful soup on the html
            soup = BeautifulSoup(html)
            # pulls the target html which holds the link of each github on that page
            text_with_link = soup.select('.v-align-middle')
            # iterates through each peice of html and will check if the regex below matches with anything.
            for link in text_with_link:
                # turning the html to a string
                link_ispy = str(link)
                # setting all the links to match
                match = re.search(r'(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-]\w+)', link_ispy)
                # checks if match is null and only passes through non null values
                if match is not None:
                    link_list.append(match.group(0))
    return link_list

In [19]:
response = requests.get(f'https://github.com/search?p=100&q=stars%3A%3E0&s=stars&type=Repositories',headers = {'user-agent': 'Codeup DS Germain'})

In [20]:
html = response.text

In [21]:
soup = BeautifulSoup(html)

In [22]:
text_with_link = soup.select('.v-align-middle')

In [23]:
for link in text_with_link:
link_ispy = str(link)

NameError: name 'link' is not defined

In [ ]:
for link in text_with_link:

In [2]:
REPOS = acquire.get_url_list()

https://github.com/twbs/bootstrap
75
waiting
continue
https://github.com/danistefanovic/build-your-own-x
225
waiting
continue
https://github.com/electron/electron
381
waiting
continue
https://github.com/gin-gonic/gin
528
waiting
continue
https://github.com/trimstray/the-book-of-secret-knowledge
681
waiting
continue
https://github.com/cdr/code-server
830
waiting
continue
https://github.com/scutan90/DeepLearning-500-questions
974
waiting
continue
https://github.com/mtdvio/every-programmer-should-know
1129
waiting
continue
https://github.com/sdmg15/Best-websites-a-programmer-should-visit
1276
waiting
continue


In [7]:
REPOS

['https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/twbs/bootstrap',
 'https://github.com/twbs/boots

In [5]:
REPO_LIST = list(set(REPOS))

In [6]:
len(REPO_LIST)

76